In [9]:
import pandas as pd
from pathlib import Path
import sys

# --- ROBUST PATH DETECTION ---
# We try multiple locations to find where 'data' is hiding
possible_paths = [
    Path("data"),           # data/ inside current folder
    Path("../data"),        # data/ in parent folder (common for notebooks)
    Path("."),              # current folder
    Path("D:/Projects/Projects Collection/Final Year Project/Tourism Application/data") # Hardcoded fallback
]

DATA_DIR = None
for path in possible_paths:
    if (path / "FinalYearProject_Survey.csv").exists():
        DATA_DIR = path
        print(f"✅ Data found in: {DATA_DIR.resolve()}")
        break

if DATA_DIR is None:
    print("❌ Error: Could not find 'FinalYearProject_Survey.csv'.")
    print("Checked locations:", [p.resolve() for p in possible_paths])
    # Stop execution if data isn't found
    sys.exit("Stopping execution: Data not found.")
# -----------------------------

# Load files
try:
    survey = pd.read_csv(DATA_DIR / "FinalYearProject_Survey.csv", low_memory=False)
    tripadvisor = pd.read_csv(DATA_DIR / "tripadvisor_hotel_reviews.csv", low_memory=False)
    # Encoding fix for Reviews.csv
    reviews = pd.read_csv(DATA_DIR / "Reviews.csv", low_memory=False, encoding='latin1')
    travel_agents = pd.read_csv(DATA_DIR / "sri_lanka_travel_agents.csv", low_memory=False)
    tourist_shops = pd.read_csv(DATA_DIR / "sri_lanka_tourist_shops.csv", low_memory=False)
    
    # Handle potentially different filename
    if (DATA_DIR / "sri_lanka_aggregated_2020_2022.csv").exists():
        aggregated = pd.read_csv(DATA_DIR / "sri_lanka_aggregated_2020_2022.csv", low_memory=False)
    else:
        aggregated = pd.read_csv(DATA_DIR / "ri_lanka_aggregated_2020_2022.csv", low_memory=False)

    enriched40 = pd.read_csv(DATA_DIR / "Tourism_dataset.csv", low_memory=False)

    # Analysis
    total_responses = len(survey)
    
    # Flexible column search for 'Nationality'
    country_col = '2. What is your country or nationality?'
    if country_col not in survey.columns:
        cols = [c for c in survey.columns if "country" in c.lower()]
        if cols: country_col = cols[0]

    domestic = survey[survey[country_col].str.contains("Sri Lanka|Sri Lankan|Srilanka|Sinhal", case=False, na=False)]
    international = survey[~survey.index.isin(domestic.index)]

    print("Total responses:", total_responses)
    print("Domestic responses:", len(domestic))
    print("International responses:", len(international))

    # Save summary
    summary = {
        "total_responses": total_responses,
        "domestic": len(domestic),
        "international": len(international)
    }
    pd.Series(summary).to_csv(DATA_DIR / "survey_summary_counts.csv")
    print("✅ Summary saved successfully.")

except Exception as e:
    print(f"❌ An error occurred during processing: {e}")

✅ Data found in: D:\Projects\Projects Collection\Final Year Project\Tourism Application\data
Total responses: 101
Domestic responses: 61
International responses: 40
✅ Summary saved successfully.


In [10]:
import pandas as pd
from pathlib import Path
import sys

# --- ROBUST PATH DETECTION ---
possible_paths = [Path("data"), Path("../data"), Path(".")]
DATA_DIR = None
for path in possible_paths:
    if (path / "Tourism_dataset.csv").exists():
        DATA_DIR = path
        break

if DATA_DIR is None:
    sys.exit("❌ Error: Could not find 'Tourism_dataset.csv'. Check your data folder.")
# -----------------------------

# Reload files
tripadvisor = pd.read_csv(DATA_DIR / "tripadvisor_hotel_reviews.csv", low_memory=False)
enriched40 = pd.read_csv(DATA_DIR / "Tourism_dataset.csv", low_memory=False)
reviews = pd.read_csv(DATA_DIR / "Reviews.csv", low_memory=False, encoding='latin1')

def standardize(df, name_cols):
    for c in name_cols:
        if c in df.columns:
            df = df.rename(columns={c: "place_name"})
            break
    
    lat_cols = [c for c in df.columns if "lat" in c.lower()]
    lon_cols = [c for c in df.columns if "lon" in c.lower() or "lng" in c.lower()]
    if lat_cols: df = df.rename(columns={lat_cols[0]: "latitude"})
    if lon_cols: df = df.rename(columns={lon_cols[0]: "longitude"})
    return df

tripadvisor = standardize(tripadvisor, name_cols=["Destination", "name", "place", "hotel_name"])
reviews = standardize(reviews, name_cols=["Location_Name", "Location", "place", "location", "name"])
enriched40 = standardize(enriched40, name_cols=["name", "place_name"])

# Clean place names
for df in (tripadvisor, reviews, enriched40):
    if "place_name" in df.columns:
        df["place_name"] = df["place_name"].astype(str).str.strip().str.replace(r"\s+", " ", regex=True).str.lower()

print("✅ Files loaded and columns standardized.")
print("TripAdvisor Head:\n", tripadvisor.head(3))

✅ Files loaded and columns standardized.
TripAdvisor Head:
                 place_name District      Timespan  \
0  attidiya bird sanctuary  colombo  3 months ago   
1  attidiya bird sanctuary  colombo  3 months ago   
2  attidiya bird sanctuary  colombo   5 years ago   

                                              Review  
0  spots scenic make ideal dwelling birds creatur...  
1      good place birdwatching different type around  
2  calm peaceful location visit time got separate...  


In [11]:
import pandas as pd
from pathlib import Path
import difflib
import sys

# --- ROBUST PATH DETECTION ---
possible_paths = [Path("data"), Path("../data"), Path(".")]
DATA_DIR = None
for path in possible_paths:
    if (path / "Tourism_dataset.csv").exists():
        DATA_DIR = path
        break

if DATA_DIR is None:
    sys.exit("❌ Error: Could not find data files.")
# -----------------------------

# Load files
tripadvisor = pd.read_csv(DATA_DIR / "tripadvisor_hotel_reviews.csv", low_memory=False)
enriched40 = pd.read_csv(DATA_DIR / "Tourism_dataset.csv", low_memory=False)
reviews = pd.read_csv(DATA_DIR / "Reviews.csv", low_memory=False, encoding='latin1')

def standardize_names(df, possible_names):
    for c in possible_names:
        if c in df.columns:
            df = df.rename(columns={c: "place_name"})
            break
    if "place_name" in df.columns:
        df["place_name"] = df["place_name"].astype(str).str.strip().str.lower()
    return df

tripadvisor = standardize_names(tripadvisor, ["Destination", "name"])
enriched40 = standardize_names(enriched40, ["name", "place_name"])

# Canonical names
canonical_names = enriched40["place_name"].dropna().unique().tolist()

# 1. Exact join
tripadvisor_exact = tripadvisor.merge(enriched40, on="place_name", how="left", suffixes=("","_enr"))

# 2. Fuzzy match
missing_mask = tripadvisor_exact["category"].isna()
print(f"Records missing match: {missing_mask.sum()}")

def get_best_match(name, choices, cutoff=0.75):
    matches = difflib.get_close_matches(name, choices, n=1, cutoff=cutoff)
    return matches[0] if matches else None

updates = []
for idx, row in tripadvisor_exact[missing_mask].iterrows():
    name = row["place_name"]
    if pd.isna(name) or name == "": continue
    match = get_best_match(name, canonical_names)
    if match:
        updates.append((idx, match))

print(f"Found {len(updates)} fuzzy matches.")

for idx, match_name in updates:
    enr_row = enriched40[enriched40["place_name"] == match_name].iloc[0]
    for col in enriched40.columns:
        tripadvisor_exact.at[idx, col] = enr_row[col]

tripadvisor_exact.to_csv(DATA_DIR / "tripadvisor_enriched_merged.csv", index=False)
print("✅ Merged tripadvisor saved.")

Records missing match: 33219
Found 1507 fuzzy matches.
✅ Merged tripadvisor saved.


In [12]:
import pandas as pd
from pathlib import Path
from textblob import TextBlob
import difflib
import sys

# --- ROBUST PATH DETECTION ---
possible_paths = [Path("data"), Path("../data"), Path(".")]
DATA_DIR = None
for path in possible_paths:
    if (path / "Tourism_dataset.csv").exists():
        DATA_DIR = path
        break

if DATA_DIR is None:
    sys.exit("❌ Error: Could not find data files.")
# -----------------------------

enriched40 = pd.read_csv(DATA_DIR / "Tourism_dataset.csv", low_memory=False)
tripadvisor_enriched = pd.read_csv(DATA_DIR / "tripadvisor_enriched_merged.csv", low_memory=False)
reviews = pd.read_csv(DATA_DIR / "Reviews.csv", low_memory=False, encoding='latin1')

if "Location_Name" in reviews.columns:
    reviews = reviews.rename(columns={"Location_Name": "place_name"})
reviews["place_name"] = reviews["place_name"].astype(str).str.strip().str.lower()
enriched40["name_clean"] = enriched40["name"].astype(str).str.strip().str.lower()

canonical_names = enriched40["name_clean"].tolist()
reviews["matched_name"] = reviews["place_name"].apply(lambda x: (difflib.get_close_matches(x, canonical_names, n=1, cutoff=0.75) or [x])[0])

agg_reviews = reviews.groupby("matched_name").agg(
    avg_rating=("Rating", "mean"),
    review_count=("Rating", "count")
).reset_index().rename(columns={"matched_name": "place_name"})

def get_sentiment(text):
    try:
        return TextBlob(str(text)).sentiment.polarity
    except:
        return 0.0

if "Review" in tripadvisor_enriched.columns:
    tripadvisor_enriched["sentiment"] = tripadvisor_enriched["Review"].apply(get_sentiment)
    agg_sent = tripadvisor_enriched.groupby("place_name").agg(
        avg_sentiment=("sentiment", "mean")
    ).reset_index()
else:
    agg_sent = pd.DataFrame(columns=["place_name", "avg_sentiment"])

master = enriched40.rename(columns={"name_clean": "place_name"})
master = master.merge(agg_reviews, on="place_name", how="left")
master = master.merge(agg_sent, on="place_name", how="left")

master["avg_rating"] = master["avg_rating"].fillna(master["avg_rating"].mean())
master["review_count"] = master["review_count"].fillna(0)
master["avg_sentiment"] = master["avg_sentiment"].fillna(0.0)

max_ven = master["vendor_count"].max() or 1
max_rev = master["review_count"].max() or 1

master["local_empowerment_signal"] = (
    0.4 * (master["vendor_count"] / max_ven) +
    0.4 * (master["review_count"] / max_rev) +
    0.2 * (pd.to_numeric(master["local_empowerment_index"], errors='coerce').fillna(0) / 10.0)
)

master.to_csv(DATA_DIR / "master_enriched_dataset.csv", index=False)
print("✅ Master dataset saved.")

✅ Master dataset saved.


In [13]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
import joblib
import sys

# --- ROBUST PATH DETECTION ---
possible_paths = [Path("data"), Path("../data"), Path(".")]
DATA_DIR = None
for path in possible_paths:
    if (path / "master_enriched_dataset.csv").exists():
        DATA_DIR = path
        break
if DATA_DIR is None:
    # Fallback checks
    if Path("master_enriched_dataset.csv").exists():
        DATA_DIR = Path(".")
    else:
        sys.exit("❌ Error: master_enriched_dataset.csv not found. Run step 04 first.")
# -----------------------------

MODEL_DIR = Path("models")
MODEL_DIR.mkdir(exist_ok=True)

print(f"Loading data from {DATA_DIR}...")
df = pd.read_csv(DATA_DIR / "master_enriched_dataset.csv")

# --- Step 2: Preprocessing ---
# Handle missing values & Target
df = df.dropna(subset=["category"])
df["category"] = df["category"].str.lower().str.strip()
df["category"] = df["category"].replace({"hperadeniya eco": "eco"})

features = [
    "eco_score", "cultural_score", "vendor_count", "local_empowerment_index",
    "accessibility_score", "danger_level", "avg_rating", "review_count", "avg_sentiment"
]

# Map crowd level to numeric
df["crowd_level_num"] = df["crowd_level"].map({"low":1, "medium":2, "high":3}).fillna(2)
features.append("crowd_level_num")

# Fill missing numeric values
for f in features:
    df[f] = pd.to_numeric(df[f], errors="coerce").fillna(df[f].median())

X = df[features]
le = LabelEncoder()
y = le.fit_transform(df["category"])

# Split Data
# Using stratify to maintain class balance
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Scale Data
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

# --- Step 3: Model Training & Comparison ---
print("🤖 Training & Evaluating models...")

# 5-Fold Cross Validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Algorithm 1: Random Forest
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf_cv_scores = cross_val_score(rf, X_train_s, y_train, cv=cv, scoring='accuracy')
rf.fit(X_train_s, y_train)
rf_pred = rf.predict(X_test_s)
rf_acc = accuracy_score(y_test, rf_pred)
print(f"   Random Forest CV Mean: {rf_cv_scores.mean():.2%} | Test Acc: {rf_acc:.2%}")

# Algorithm 2: Logistic Regression
lr = LogisticRegression(max_iter=1000, random_state=42)
lr_cv_scores = cross_val_score(lr, X_train_s, y_train, cv=cv, scoring='accuracy')
lr.fit(X_train_s, y_train)
lr_pred = lr.predict(X_test_s)
lr_acc = accuracy_score(y_test, lr_pred)
print(f"   Logistic Regression CV Mean: {lr_cv_scores.mean():.2%} | Test Acc: {lr_acc:.2%}")

# Select Best Model
if rf_acc >= lr_acc:
    best_model = rf
    best_name = "Random Forest"
    best_acc = rf_acc
    final_preds = rf_pred
else:
    best_model = lr
    best_name = "Logistic Regression"
    best_acc = lr_acc
    final_preds = lr_pred

print(f"🏆 Best Model Selected: {best_name}")

# Calculate Confusion Matrix for the best model
cm = confusion_matrix(y_test, final_preds)

# --- Step 4: Explainable AI Preparation ---
# Save a background dataset for SHAP (important for the app!)
# We take a sample of the scaled training data (e.g., 100 rows)
# This makes the explainer efficient.
if X_train_s.shape[0] > 100:
    background_indices = np.random.choice(X_train_s.shape[0], 100, replace=False)
    background_data = X_train_s[background_indices]
else:
    background_data = X_train_s

# --- Save Everything for the App ---
joblib.dump(best_model, MODEL_DIR / "best_model.pkl")
joblib.dump(scaler, MODEL_DIR / "scaler.pkl")
joblib.dump(le, MODEL_DIR / "label_encoder.pkl")
joblib.dump(background_data, MODEL_DIR / "background_data.pkl") # Saved for SHAP

# Save metrics dict for the "Model Performance" section
metrics = {
    "rf_accuracy": rf_acc,
    "lr_accuracy": lr_acc,
    "rf_cv_score": rf_cv_scores.mean(),
    "lr_cv_score": lr_cv_scores.mean(),
    "best_model_name": best_name,
    "confusion_matrix": cm,
    "classes": le.classes_
}
joblib.dump(metrics, MODEL_DIR / "model_metrics.pkl")

print(f"✅ Models, XAI background data, and metrics saved to {MODEL_DIR}")

Loading data from ..\data...
🤖 Training & Evaluating models...


d:\Projects\Projects Collection\Final Year Project\Tourism Application\venv\Lib\site-packages\sklearn\model_selection\_split.py:813: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


   Random Forest CV Mean: 90.95% | Test Acc: 87.50%
   Logistic Regression CV Mean: 90.95% | Test Acc: 75.00%
🏆 Best Model Selected: Random Forest
✅ Models, XAI background data, and metrics saved to models


d:\Projects\Projects Collection\Final Year Project\Tourism Application\venv\Lib\site-packages\sklearn\model_selection\_split.py:813: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
